In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wdtools import *

In [3]:
setID = 'Set008'; nm_to_add = 5402

In [24]:
gdf_file = f'{outpath}//final//mapped_wd_{setID}.shp'

In [25]:
os.path.isfile(gdf_file)

False

In [35]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add, raw=False)

In [36]:
wd.county.unique()

array(['Lake', 'Marion'], dtype=object)

In [20]:
len(wd.wetdet_delin_number.unique())

295

In [5]:
matched = gpd.read_file(f'{outpath}\\matched\\matched_records_Set008.shp')

In [7]:
file = fr'{outpath}\to_review\re_mapping_{setID}.txt' 
with open(file) as f: 
    remapIDs = f.readlines()

In [8]:
unmatched = pd.read_csv(fr'{outpath}\to_review\unmatched_df_{setID}_2.csv')

In [10]:
wd.notes.unique()

array([nan, 'Many', 'Partial', 'ROW', 'ROW, Partial'], dtype=object)

In [11]:
partialIDs = list(wd[~wd.notes.isnull()].wetdet_delin_number.unique())

In [13]:
len(partialIDs)

83

In [14]:
unmatchedIDs = list(unmatched.wetdet_delin_number.unique())

In [16]:
issues = pd.read_excel(fr'{outpath}\to_review\{setID}_Mapping_Issues.xlsx')

In [17]:
issueIDs = list(issues.wetdet_delin_number.unique())

In [18]:
qaqcIDs = unique(remapIDs[0].split(', ') + partialIDs + unmatchedIDs + issueIDs)

In [40]:
len(qaqcIDs)

109

In [21]:
len([wid for wid in matched.wdID.unique() if wid not in qaqcIDs])

186

In [22]:
check_cnt = wd[wd.wetdet_delin_number.isin(qaqcIDs)][['county', 'wetdet_delin_number']].groupby(['wetdet_delin_number']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'wdID', 'county':'count'})

In [23]:
check_cnt[check_cnt['count']>1]

,wdID,count


In [41]:
qaqc_df = wd[wd.wetdet_delin_number.isin(qaqcIDs)][['county', 'wetdet_delin_number']].groupby(['county']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'QAQC_count'})

In [42]:
qaqc_df

,county,QAQC_count
0,Lake,21
1,Marion,88


In [43]:
gdf = update_QAQC_data(setID=setID, wd=wd, 
                        totcol='P2_2008_16', 
                        qaqc_col='P2_ToMap', 
                        export=True)

In [37]:
gdf.columns

Index(['County', 'Note', 'P1_2017_22', 'P1_ToMap', 'P2_2008_16', 'P2_ToMap',
       'P3_1990_07', 'P3_ToMap', 'Tot_ToMap', 'Tot_Count', 'QAQC_Ratio',
       'Auto_Comp', 'geometry'],
      dtype='object')

In [44]:
gdf[gdf.County.isin(wd.county.unique())][['County','P2_2008_16', 'P2_ToMap']]

,County,P2_2008_16,P2_ToMap
5,Lake,33,21
20,Marion,262,88


In [3]:
setID = 'Set007'; nm_to_add = 4972

In [4]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add, raw=False)

In [16]:
mapped = gpd.read_file(f'{outpath}//final//mapped_wd_{setID}.shp')

file = outpath + f'\\to_review\\re_mapping_{setID}.txt'
with open(file) as f:
    remapIDs = f.readlines()

In [6]:
wd.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [5]:
len(wd.wetdet_delin_number.unique())

299

partialIDs = list(wd[wd.notes != ''].wetdet_delin_number.unique())

unmatched = pd.read_csv(outpath+f'\\to_review\\unmatched_df_{setID}_2.csv')

unmatched.columns

unmatchedIDs = list(unmatched.wetdet_delin_number.unique())

issues = pd.read_excel(outpath + f'\\to_review\\{setID}_Mapping_Issues.xlsx')

issueIDs = list(issues.wetdet_delin_number.unique())

In [20]:
qaqcIDs = wd[~wd.wetdet_delin_number.isin(mapped.wdID.unique())].wetdet_delin_number.unique() #remapIDs[0].split(', ') + partialIDs + unmatchedIDs + issueIDs

In [21]:
len(qaqcIDs)

26

In [22]:
check_cnt = wd[wd.wetdet_delin_number.isin(qaqcIDs)][['county', 'wetdet_delin_number']].groupby(['wetdet_delin_number']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'wdID', 'county':'count'})

In [23]:
check_cnt[check_cnt['count']>1]

,wdID,count


In [26]:
gdf = update_QAQC_data(setID=setID, wd=wd, 
                        totcol='P2_2008_16', 
                        qaqc_col='P2_ToMap', 
                        export=True)

In [45]:
os.remove(inpath+"\\reporting\\WD_Counts.zip")

In [46]:
file_name_list = glob.glob(inpath+"\\reporting\\WD_Counts.*")
zip_file_name = inpath+"\\reporting\\WD_Counts.zip"
file_compress(file_name_list, zip_file_name)

 *** Input File name passed for zipping - ['L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.cpg', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.dbf', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.prj', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.shp', 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\reporting\\WD_Counts.shx']
 *** out_zip_file is - L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping\reporting\WD_Counts.zip
 *** Processing file L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping\reporting\WD_Counts.cpg
 *** Processing file L:\NaturalResources\Wetlands\Loc

ignore below codes

In [120]:
qaqc_df = wd[wd.wetdet_delin_number.isin(qaqcIDs)][['county', 'wetdet_delin_number']].groupby(['county']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'QAQC_count'})

In [121]:
qaqc_df

,county,QAQC_count
0,Clatsop,91
1,Deschutes,26


In [37]:
total_df = wd[['county', 'wetdet_delin_number']].groupby(['county']).agg(lambda x: x.nunique()).reset_index().rename(columns={'wetdet_delin_number':'total_count'})

In [38]:
total_df

,county,total_count
0,Clatsop,201
1,Deschutes,98


In [31]:
gdf = gpd.read_file(inpath+'\\reporting\\WD_Counts.shp')

In [34]:
gdf.columns

Index(['County', 'Note', 'P1_2017_22', 'P1_ToMap', 'P2_2008_16', 'P2_ToMap',
       'P3_1990_07', 'P3_ToMap', 'Tot_ToMap', 'Tot_Count', 'QAQC_Ratio',
       'Auto_Comp', 'geometry'],
      dtype='object')

In [58]:
sel = qaqc_df.county.unique()

In [52]:
total_df.loc[total_df.county.isin(sel), 
             ['county', 'total_count']]

,county,total_count
0,Clatsop,201
1,Deschutes,98


In [54]:
gdf.loc[gdf.County.isin(sel), 
        ['County', 'P2_2008_16', 'P2_ToMap']]

,County,P2_2008_16,P2_ToMap
9,Deschutes,132,66
35,Clatsop,275,137


In [44]:
wdcnt_dict = total_df.set_index('county').to_dict(orient='dict')['total_count']

In [122]:
qccnt_dict = qaqc_df.set_index('county').to_dict(orient='dict')['QAQC_count']

In [123]:
qccnt_dict

{'Clatsop': 91, 'Deschutes': 26}

In [55]:
gdf.loc[gdf.County.isin(sel), 'P2_2008_16'] = gdf[gdf.County.isin(sel)].County.map(wdcnt_dict)

In [124]:
gdf.loc[gdf.County.isin(sel), 'P2_ToMap'] = gdf[gdf.County.isin(sel)].County.map(qccnt_dict)

In [125]:
gdf.loc[gdf.County.isin(sel), 
        ['County', 'P2_2008_16', 'P2_ToMap']]

,County,P2_2008_16,P2_ToMap
9,Deschutes,98,26
35,Clatsop,201,91


In [126]:
gdf.loc[:, 'Tot_ToMap'] =  gdf.P1_ToMap + gdf.P2_ToMap + gdf.P3_ToMap
gdf.loc[:, 'Tot_Count'] =  gdf.P1_2017_22 + gdf.P2_2008_16 + gdf.P3_1990_07
gdf.loc[:, 'QAQC_Ratio'] = gdf.Tot_ToMap / gdf.Tot_Count
gdf.loc[:, 'Auto_Comp'] = gdf.Tot_Count - gdf.Tot_ToMap

In [127]:
gdf.to_file(inpath+'\\reporting\\WD_Counts.shp')

In [132]:
test.loc[test.County.isin(sel), 
        ['County', 'P2_2008_16', 'P2_ToMap']]

,County,P2_2008_16,P2_ToMap
9,Deschutes,98,26
35,Clatsop,201,91
